In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config

import twelvedata
from twelvedata import TDClient


In [2]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [3]:
# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="bailish_estate"   
)

mycursor = mydb.cursor()


# Date range for historical data (funky start date due to downloading first month as test batch)
focalpoint = datetime.now()
end_date = focalpoint - timedelta(days=3) 
start_date = end_date - timedelta(days=1)  

# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000  

In [4]:


# Fetch time series data and indicators
NVDA_time_series = td.time_series(
    symbol=symbol,
    interval=interval,
    outputsize=outputsize,
    start_date=start_date.strftime("%Y-%m-%d"),
    end_date=end_date.strftime("%Y-%m-%d"),
).with_macd().with_ema().with_rsi().with_macd(fast_period=10).with_stoch().as_json()



In [5]:
nvda_time_series_df = pd.DataFrame(NVDA_time_series)
nvda_time_series_df['symbol'] = symbol

In [6]:
print(nvda_time_series_df)
nvda_time_series_df.to_csv('nvda_time_series_df.csv', index=False)

                datetime       open       high        low      close    volume   macd_1  ...        ema       rsi   macd_2 macd_signal_2 macd_hist_2    slow_k    slow_d
0    2024-08-01 15:59:00  109.02000  109.49000  108.90000  109.33000   3694194  0.20647  ...  108.80605  65.41440  0.25795       0.15051     0.10745  91.06168  87.40443
1    2024-08-01 15:58:00  109.00000  109.27000  108.86250  109.02000   2367292  0.16130  ...  108.67506  61.23285  0.19975       0.12365     0.07610  84.07643  81.65385
2    2024-08-01 15:57:00  108.79000  109.10000  108.74000  108.98000   1759321  0.13287  ...  108.58883  60.66312  0.16346       0.10462     0.05884  87.07519  69.27467
3    2024-08-01 15:56:00  108.40000  108.87000  108.38000  108.78500   1580931  0.09872  ...  108.49103  57.85912  0.11834       0.08991     0.02843  73.80992  52.72132
4    2024-08-01 15:55:00  108.25000  108.75970  108.16000  108.39000   2093667  0.07355  ...  108.41754  51.33405  0.08441       0.08280     0.00160  46.93

In [7]:
sql_insert = """
INSERT INTO nvda_db (datetime, open, high, low, close, volume, macd_1, macd_signal_1, macd_hist_1, ema, rsi, macd_2, macd_signal_2, macd_hist_2, slow_k, slow_d)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)  
"""

# Ensure data types are compatible
nvda_time_series_df["volume"] = nvda_time_series_df["volume"].astype(int)  # Convert to int if needed

# Convert DataFrame rows to tuples
data_to_insert = [tuple(x) for x in nvda_time_series_df.to_numpy()]

# Execute the insert query
mycursor.executemany(sql_insert, data_to_insert)
mydb.commit()

In [8]:
nvda_time_series_df['symbol'] = "NVDA"